In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
#Input headers
headers = {'User-Agent':'}

#User to input desired stock ticker symbols, which are saved onto a list
tickers = list(str(input('Input ticker symbols separated by a space: ')).upper().split())

#Initialize empty list to save data obtained from following for-loop 
all_data = []

#Loop through all ticker symbols
for ticker in tickers:
    try: 
        #Relevant url with the data to be scraped
        url = f'https://finance.yahoo.com/quote/{ticker}'
        
        #Access the url and save response
        response = requests.get(url,headers=headers)
        
        #BS4 object
        soup = BeautifulSoup(response.text,'html.parser')
        
        #Find the html tags for the summary table 
        table = soup.find('div',{'class':'Bxz(bb) D(ib) Va(t) Mih(250px)!--lgv2 W(100%) Mt(-6px) Mt(0px)--mobp Mt(0px)--mobl W(50%)!--lgv2 Mend(20px)!--lgv2 Pend(10px)!--lgv2'})
        
        rows=[]
        #Loop through all rows from 'tr' tags
        for row in table.find_all('tr'):
            cells=[]
            #Loop through all columns from 'td' tags - aka cells
            for cell in row.find_all('td'):
                #Append each cell in the foregoing empty 'cells' list
                cells.append(cell.text)
            #Append all cells onto the rows empty list
            rows.append(cells)
       #Create a Pandas dataframe from the rows
        df = pd.DataFrame(rows)
        
        #Define columns and add a column with ticker
        df.columns = ['Metric', 'Data']
        df['Ticker'] = ticker
        
        #The df is in long form - we need transform it to wide format
        data = pd.pivot(df, index='Ticker', columns='Metric', values='Data')
        
        #Drop empty columns
        data = data.dropna(axis=1,how='all',inplace=False)
    
        #Append data onto a list of all data
        all_data.append(data)
        print('Done: ', ticker)
    
    #Exception to handle an invalid ticker symbol 
    except AttributeError:
        print(f'{ticker} does not exist')

Input ticker symbols separated by a space: goog amzn tsla brk.a nvda meta wmt nflx aapl msft
Done:  GOOG
Done:  AMZN
Done:  TSLA
Done:  BRK.A
Done:  NVDA
Done:  META
Done:  WMT
Done:  NFLX
Done:  AAPL
Done:  MSFT


In [4]:
#Concatenate all dataframes in the list of dataframes into one dataframe
all_data_df = pd.concat([df for df in all_data], axis=0)

#Save to csv
all_data_df.to_csv('/users/solomonmuthui/Desktop/Py/summary_table_YF.csv')
all_data_df

Metric,1y Target Est,52 Week Range,Ask,Avg. Volume,Beta (5Y Monthly),Bid,Day's Range,EPS (TTM),Earnings Date,Ex-Dividend Date,Forward Dividend & Yield,Market Cap,Open,PE Ratio (TTM),Previous Close,Volume
Ticker,,,,,,,,,,,,,,,,
GOOG,148.95,102.21 - 152.10,0.00 x 800,"25,635,427",1.09,102.71 x 800,103.31 - 106.20,5.27,"Oct 24, 2022 - Oct 28, 2022",N/A,N/A (N/A),1.375T,105.01,19.72,105.87,"26,398,063"
AMZN,171.18,101.26 - 188.11,0.00 x 900,"59,646,545",1.33,0.00 x 800,125.50 - 130.37,1.12,"Oct 26, 2022 - Oct 31, 2022",N/A,N/A (N/A),1.286T,127.38,112.75,128.55,"52,887,196"
TSLA,305.10,206.86 - 414.50,299.92 x 1200,"78,820,261",2.19,299.62 x 1000,300.72 - 309.12,2.78,"Oct 18, 2022 - Oct 24, 2022",N/A,N/A (N/A),948.219B,301.83,109.26,302.61,"64,795,523"
BRK.A,N/A,undefined - undefined,N/A x N/A,N/A,N/A,N/A x N/A,N/A - N/A,N/A,"Aug 02, 2018 - Aug 06, 2018",N/A,N/A (N/A),N/A,N/A,N/A,N/A,N/A
NVDA,201.60,127.90 - 346.47,127.29 x 1100,"54,143,766",1.69,126.95 x 1300,128.01 - 132.32,3.73,"Nov 15, 2022 - Nov 21, 2022","Sep 07, 2022",0.16 (0.12%),322.191B,130.15,34.67,131.28,"52,362,516"
META,226.23,148.70 - 371.41,148.40 x 1100,"28,653,927",1.30,148.11 x 1400,148.70 - 154.20,10.81,"Oct 24, 2022 - Oct 28, 2022",N/A,N/A (N/A),401.923B,149.80,13.83,151.47,"33,314,973"
WMT,151.89,117.27 - 160.77,132.58 x 900,"7,979,422",0.53,131.82 x 1400,132.97 - 135.15,5.00,"Nov 15, 2022","Dec 08, 2022",2.24 (1.68%),362.27B,135.05,26.69,134.92,"5,937,184"
NFLX,244.73,162.71 - 700.99,231.99 x 800,"9,613,256",1.34,231.60 x 1100,229.10 - 242.29,11.82,"Oct 18, 2022",N/A,N/A (N/A),104.675B,230.47,19.91,224.12,"19,454,122"
AAPL,181.77,129.04 - 182.94,150.94 x 900,"73,894,148",1.23,150.60 x 900,151.38 - 155.24,6.05,"Oct 26, 2022 - Oct 31, 2022","Aug 05, 2022",0.92 (0.60%),2.496T,154.65,25.19,155.31,"90,481,110"
